# 0 环境配置

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2018)

from keras.optimizers import *
from model.coremodel import coremodel
from dataload.my_image import MyImageDataGenerator
from loss import myloss
from metric import diff_accuracy, count_accuracy
print("import done")

Using TensorFlow backend.


import done


# 1 预设参数

In [4]:
# test
basedir = "/home/ubuntu/JupyterNotebook/columncount/data"
basemodel_name = 'inception_v3'
model_image_size = (480, 640)
train_samples_epoch = 120
train_batch_size = 6
steps_train = train_samples_epoch//train_batch_size + 1
print("steps_train = {}".format(steps_train))
valid_samples_epoch = 39
valid_batch_size = 20
steps_valid = valid_samples_epoch//valid_batch_size + 1
print("steps_valid = {}".format(steps_valid))

steps_train = 21
steps_valid = 2


In [18]:
basedir = "/home/ubuntu/JupyterNotebook/columncount/data"
basemodel_name = 'inception_v3'
model_image_size = (480, 640)
train_samples_epoch = 20000
train_batch_size = 64
steps_train = train_samples_epoch//train_batch_size + 1
print("steps_train = {}".format(steps_train))
valid_samples_epoch = 2000
valid_batch_size = 64
steps_valid = valid_samples_epoch//valid_batch_size + 1
print("steps_valid = {}".format(steps_valid))

steps_train = 313
steps_valid = 32


In [5]:
# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError

In [6]:
modelname = basemodel_name + '_20180123'

# 2 数据生成

In [7]:
train_gen = MyImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=(0.85, 1.1),
    channel_shift_range=10.,
)

gen = MyImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.myflow_from_directory(os.path.join(basedir, 'train'),  model_image_size,
                                                shuffle=True, batch_size=train_batch_size)

valid_generator = gen.myflow_from_directory(os.path.join(basedir, 'valid'),  model_image_size,
                                          shuffle=True, batch_size=valid_batch_size)

Found 20 images.
Found 20 images.


# 3 搭建模型

In [8]:
model = coremodel(basemodel_name, model_image_size)

# 4 训练模型

In [9]:
# modify myloss, use pure version
model.compile(optimizer=Adam(lr=1e-4), 
              loss=myloss(diff_weight=0.2, count_weight=0.8),
              metrics=[diff_accuracy(), count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/50
21/21 [==============================] - 16s 780ms/step - loss: 3.7938 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.0240 - val_loss: 0.5566 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.0500
Epoch 2/50
21/21 [==============================] - 9s 420ms/step - loss: 0.6266 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.0952 - val_loss: 0.7728 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.1000
Epoch 3/50
21/21 [==============================] - 9s 414ms/step - loss: 0.5687 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.1346 - val_loss: 1.6056 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2000
Epoch 4/50
21/21 [==============================] - 9s 431ms/step - loss: 0.4967 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.1361 - val_loss: 0.8199 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2000
Epoch 5/50
21/21 [==============================] - 9s 418ms/step - loss: 0.2927 - _diff_accuracy: 0.0400 - _count_accuracy: 0.

In [10]:
# modify myloss, use pure version
model.compile(optimizer=Adam(lr=1e-5), 
              loss=myloss(diff_weight=0.2, count_weight=0.8),
              metrics=[diff_accuracy(), count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/50
21/21 [==============================] - 15s 720ms/step - loss: 0.0264 - _diff_accuracy: 0.9296 - _count_accuracy: 0.9920 - val_loss: 0.0203 - val__diff_accuracy: 1.0000 - val__count_accuracy: 1.0000
Epoch 2/50
21/21 [==============================] - 9s 428ms/step - loss: 0.0222 - _diff_accuracy: 0.9689 - _count_accuracy: 1.0000 - val_loss: 0.0199 - val__diff_accuracy: 1.0000 - val__count_accuracy: 0.9500
Epoch 3/50
21/21 [==============================] - 9s 447ms/step - loss: 0.0197 - _diff_accuracy: 0.9688 - _count_accuracy: 0.9768 - val_loss: 0.0209 - val__diff_accuracy: 1.0000 - val__count_accuracy: 0.9500
Epoch 4/50
21/21 [==============================] - 9s 411ms/step - loss: 0.0238 - _diff_accuracy: 0.9840 - _count_accuracy: 0.9920 - val_loss: 0.0171 - val__diff_accuracy: 1.0000 - val__count_accuracy: 0.9500
Epoch 5/50
21/21 [==============================] - 8s 404ms/step - loss: 0.0240 - _diff_accuracy: 0.9296 - _count_accuracy: 0.9768 - val_loss: 0.0196 - val__d

In [7]:
# don't delete, mean_squared_error for count loss
model.compile(optimizer=Adam(lr=1e-4), 
              loss=myloss(diff_weight=0.2, count_weight=0.8),
              metrics=[diff_accuracy(), count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/50
21/21 [==============================] - 16s 770ms/step - loss: 3.7286 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2384 - val_loss: 1.9715 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2069
Epoch 2/50
21/21 [==============================] - 9s 432ms/step - loss: 0.6259 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2611 - val_loss: 7.2440 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2069
Epoch 3/50
21/21 [==============================] - 10s 455ms/step - loss: 0.4829 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.3003 - val_loss: 3.1262 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2069
Epoch 4/50
21/21 [==============================] - 9s 441ms/step - loss: 0.5550 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2718 - val_loss: 4.3679 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.1897
Epoch 5/50
21/21 [==============================] - 9s 420ms/step - loss: 0.3153 - _diff_accuracy: 0.0080 - _count_accuracy: 0

In [8]:
# don't delete, original version, using int_regression_loss and coeff reducing countloss
model.compile(optimizer=Adam(lr=1e-4), 
              loss=myloss(diff_weight=0.2, count_weight=0.8),
              metrics=[diff_accuracy(), count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/50
21/21 [==============================] - 16s 784ms/step - loss: 2.9395 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2073 - val_loss: 1.1085 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.1897
Epoch 2/50
21/21 [==============================] - 9s 440ms/step - loss: 0.4493 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2148 - val_loss: 0.5342 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2069
Epoch 3/50
21/21 [==============================] - 9s 447ms/step - loss: 0.2402 - _diff_accuracy: 0.1752 - _count_accuracy: 0.3033 - val_loss: 0.3866 - val__diff_accuracy: 0.3448 - val__count_accuracy: 0.2586
Epoch 4/50
21/21 [==============================] - 9s 442ms/step - loss: 0.2543 - _diff_accuracy: 0.5071 - _count_accuracy: 0.3025 - val_loss: 0.3192 - val__diff_accuracy: 0.8103 - val__count_accuracy: 0.2414
Epoch 5/50
21/21 [==============================] - 9s 440ms/step - loss: 0.1272 - _diff_accuracy: 0.7864 - _count_accuracy: 0.4128 - val_loss:

In [7]:
# don't delete, modified version, using mask to count countloss and reduce thresold to 0.0025
model.compile(optimizer=Adam(lr=1e-4), 
              loss=myloss(diff_weight=0.2, count_weight=0.8),
              metrics=[diff_accuracy(), count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/50
21/21 [==============================] - 16s 786ms/step - loss: 3.4979 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2153 - val_loss: 1.6997 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2069
Epoch 2/50
21/21 [==============================] - 9s 421ms/step - loss: 0.5871 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2219 - val_loss: 3.4440 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2069
Epoch 3/50
21/21 [==============================] - 9s 434ms/step - loss: 0.5340 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2927 - val_loss: 0.9266 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.2069
Epoch 4/50
21/21 [==============================] - 9s 451ms/step - loss: 0.4538 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.2962 - val_loss: 2.6640 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.1897
Epoch 5/50
21/21 [==============================] - 9s 450ms/step - loss: 0.3280 - _diff_accuracy: 0.0632 - _count_accuracy: 0.

In [8]:
model.save("savemodel/"+modelname+".h5")
print("model saved!")

model saved!


## （2） epoch 2，学习率1e-5，使用RMSprop

In [ ]:
model.compile(optimizer=RMSprop(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1,
                   validation_data=valid_generator, validation_steps=steps_valid_sample)

In [ ]:
model.save("models/"+modelname+"-2.h5")
print("model saved!")

## （3） epoch 3，学习率1e-6，使用RMSprop

In [ ]:
model.compile(optimizer=RMSprop(lr=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1,
                   validation_data=valid_generator, validation_steps=steps_valid_sample)

In [ ]:
model.save("models/"+modelname+"-3.h5")
print("model saved!")

## （4） epoch 4，学习率1e-6，使用RMSprop

In [ ]:
model.compile(optimizer=RMSprop(lr=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1,
                   validation_data=valid_generator, validation_steps=steps_valid_sample)

In [ ]:
model.save("models/"+modelname+"-4.h5")
print("model saved!")

# 5 测试模型

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2018)

from keras.optimizers import *
from model.coremodel import coremodel
from dataload.my_image import MyImageDataGenerator
from loss import myloss
from metric import diff_accuracy, count_accuracy

import tensorflow as tf
from keras.models import load_model
print("import done")

# test
basedir = "/home/ubuntu/JupyterNotebook/columncount/data"
basemodel_name = 'inception_v3'
model_image_size = (480, 640)
# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError


modelname = basemodel_name + '_20180122'

model = coremodel(basemodel_name, model_image_size)
model.load_weights("savemodel/"+modelname+".h5")

Using TensorFlow backend.


import done
steps_train = 21
steps_valid = 11
Found 20 images.
Found 20 images.
Found 20 images.


In [19]:
gen = MyImageDataGenerator(preprocessing_function=preprocess_input)
test_batch_size = 1
test_generator = gen.myflow_from_directory(os.path.join(basedir,"test"),  model_image_size,
                                         shuffle=False, batch_size=test_batch_size)
steps_test = test_generator.samples

Found 26 images.


In [12]:
from postprocess import postprocess

In [17]:
pred = model.predict_generator(test_generator, steps=steps_test)

In [10]:
output = open("pred.npy", 'wb')

In [11]:
np.save(output, pred)

In [12]:
output2 = open("pred.npy", 'rb')

In [13]:
pred = np.load(output2)

In [20]:
model.evaluate_generator(test_generator)

[3.2839885644363958, 0.9230769230769231, 0.07692307692307693]

In [21]:
for x in pred:
    print(x)
    out,out1,out2 = postprocess(x)
    
    print(out)
    print(out1)
    print(out2)

[ 0.00191854  0.00051446  0.00061891  0.00314093  0.00839827  0.00230077
  0.00072997  0.02627406  0.04819659  0.01313617  0.05914733  0.05008034
 -0.03089805  0.03582655]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
False
[ 0.00200388  0.00173123  0.00118879  0.00390543  0.01935062  0.00411487
  0.00136841 -0.06444988 -0.02496949  1.0679735   1.0667675   1.0052658
  1.0383526   1.0452452 ]
[0 0 0 0 0 0 0]
[0 0 1 1 1 1 1]
False
[2.5290546e-03 5.1362501e-03 1.9294573e-03 5.5546826e-03 1.5303645e-02
 6.8541118e-03 2.3853637e-03 9.2744172e-01 9.4058186e-01 9.7306013e-01
 1.0314002e+00 9.1946715e-01 1.9979608e+00 2.0549788e+00]
[0 0 0 0 0 0 0]
[1 1 1 1 1 2 2]
False
[3.3434040e-03 1.1625540e-02 5.6537050e-03 8.3015012e-03 5.7942409e-02
 1.6781360e-02 2.7471846e-03 1.9545995e+00 1.8976123e+00 9.7533202e-01
 9.6496820e-01 1.8861125e+00 1.9644408e+00 3.1271088e+00]
[0 0 0 0 0 0 0]
[2 2 1 1 2 2 3]
False
[9.4842445e-03 1.4539719e-02 2.3349155e-02 2.3961572e-02 9.0636718e-01
 2.2620045e-01 2.9632866e-03 2.095